# TTRPG LLM Playground Runner

This notebook runs the fine-tuning pipeline on Google Colab.

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. Setup Secrets (API Keys)
import os
from google.colab import userdata

# Set OpenAI Key for synthetic data generation
# Add 'OPENAI_API_KEY' to your Colab Secrets (Key icon on the left sidebar)
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except Exception as e:
    print("Warning: OPENAI_API_KEY secret not found. Synthetic generation step will use mocks.")

In [ ]:
# 3. Setup Workspace

# Define paths
REPO_URL = "https://github.com/riefer02/trpg-llm-playground.git" # Updated automatically
PROJECT_DIR = "/content/trpg-llm-playground/llm-playground"

# Clone or pull repo
if not os.path.exists("/content/trpg-llm-playground"):
    !git clone {REPO_URL}
else:
    %cd /content/trpg-llm-playground
    !git pull

%cd {PROJECT_DIR}

In [ ]:
# 4. Install Dependencies
!pip install -r requirements.txt
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# 5. Prepare Data
# Configure paths in config/synthetic_generic.yaml first!

# 1. Ingest PDF -> raw JSON
!python -m src.data.ingest_pdf --config config/synthetic_generic.yaml

# 2. Generate Synthetic Data -> JSONL (Uses OpenAI API)
!python -m src.data.generate_synthetic --config config/synthetic_generic.yaml

In [ ]:
# 6. Run Training
!python -m src.training.finetune_lora --config config/rpg_finetune.yaml

In [ ]:
# 7. Evaluate
!python -m src.training.evaluate --config config/rpg_finetune.yaml